In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.optim as optim
from PIL import Image
import cv2


In [ ]:
!pip install pretrainedmodels

     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60966 sha256=19cd9603a084297c64162e7fa3ba5ee229961244d574576ece850b5d69ebb5d5
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [ ]:
import pretrainedmodels

In [3]:
image_path = "drive/MyDrive/Animal_Breed/TRAIN (1)/"
train_data_transform = transforms.Compose([
        transforms.Resize((331,331),interpolation = torchvision.transforms.InterpolationMode.BICUBIC),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(20),
        # transforms.RandomRotation(20),
        transforms.ToTensor(),
        # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

train_dataset = ImageFolder(image_path, transform = train_data_transform)

In [4]:
val_path = "drive/MyDrive/Animal_Breed/VAL/"
val_data_transform = transforms.Compose([
    transforms.Resize((331,331),interpolation = torchvision.transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
val_dataset = ImageFolder(val_path, transform = val_data_transform)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, 32, 
                                             shuffle=True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(val_dataset, 1, shuffle = False, )

In [ ]:
model_name = 'nasnetalarge' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/nasnetalarge-a1897284.pth" to /root/.cache/torch/hub/checkpoints/nasnetalarge-a1897284.pth


In [ ]:
model.last_linear = nn.Linear(4032,37,bias = True)

In [16]:
inception = models.inception_v3(pretrained=True)
inception.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [17]:
inception.fc = nn.Sequential(nn.Linear(2048,37,bias = True),)
                            #  nn.ReLU(inplace = True),
                            #  nn.Dropout(0.25),
                            #  nn.Linear(1024,37)
                            #  )
                            #  nn.Hardswish(),
                            #  nn.Dropout(0.2,inplace = True),
                            #  nn.Linear(1280,37,bias = True))
inception.aux_logits = False

In [18]:
inception = inception.cuda()

In [19]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

device = get_default_device()
print(device)

cuda


In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inception.parameters(), lr = 0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=1, verbose=True)

In [21]:
def train(model,dataloader,validloader,criterion,optimizer,epochs=50,scheduler=scheduler):
    max_valid_acc = 0
    train_acc,val_acc = 0,0
    for e in range(epochs):
        train_loss = 0.0
        model.train()     # Optional when not using Model Specific layer
        for data, labels in dataloader:
#             print(labels.shape[0])
            data, labels = data.to(device), labels.to(device)
#             print(data.shape)
            optimizer.zero_grad()
            target = model(data)
#             print(target.shape)
            loss = criterion(target.float(),labels.long())
            # loss.requires_grad = True
            loss.backward()
            optimizer.step()
            train_loss = loss.item() * data.size(0)
            train_acc += torch.sum((torch.max(target, 1)[1] == labels.data),0)
            
        valid_loss = 0.0
        model.eval()     # Optional when not using Model Specific layer
        for data, labels in validloader:
            
            data, labels = data.to(device), labels.to(device)

            target = model(data)
            loss = criterion(target.float(),labels.long())
            # loss.requires_grad = True
            valid_loss = loss.item() * data.size(0)
            val_acc += torch.sum((torch.max(target, 1)[1] == labels.data),0)
            
        
        print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(dataloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
        print("Validation Accuracy ... :",val_acc/(len(validloader)))
        print("Train Accuracy ... :",train_acc/len(train_dataset))
        
        if val_acc > max_valid_acc:
            print(f'Validation Acc Increased({max_valid_acc:.6f}--->{val_acc:.6f}) \t Saving The Model')
            max_valid_acc = val_acc
            # Saving State Dict
            torch.save(model.state_dict(), 'HIGH_ACCURACY.pth')
        scheduler.step(val_acc)
        train_acc = 0
        val_acc = 0
        torch.cuda.empty_cache()
    return model

In [23]:
train(inception,train_loader,val_loader,criterion,optimizer,10)
print("Done!!")

Epoch 1 		 Training Loss: 0.11903530059100913 		 Validation Loss: 7.539572524992217e-06
Validation Accuracy ... : tensor(0.9129, device='cuda:0')
Train Accuracy ... : tensor(0.7213, device='cuda:0')
Validation Acc Increased(0.000000--->1352.000000) 	 Saving The Model
Epoch 2 		 Training Loss: 0.017908088166079077 		 Validation Loss: 6.464453996394954e-06
Validation Accuracy ... : tensor(0.9190, device='cuda:0')
Train Accuracy ... : tensor(0.9172, device='cuda:0')
Validation Acc Increased(1352.000000--->1361.000000) 	 Saving The Model
Epoch 3 		 Training Loss: 0.03140674134810194 		 Validation Loss: 3.4237439586333853e-06
Validation Accuracy ... : tensor(0.9298, device='cuda:0')
Train Accuracy ... : tensor(0.9500, device='cuda:0')
Validation Acc Increased(1361.000000--->1377.000000) 	 Saving The Model
Epoch 4 		 Training Loss: 0.022248744535789216 		 Validation Loss: 6.785320877716961e-07
Validation Accuracy ... : tensor(0.9136, device='cuda:0')
Train Accuracy ... : tensor(0.9665, devic